In [1]:
#!/usr/bin/env python

# --------------------------------------------------------
# Fast R-CNN
# Copyright (c) 2015 Microsoft
# Licensed under The MIT License [see LICENSE for details]
# Written by Ross Girshick
# --------------------------------------------------------

import menpofit
import menpofit
import menpo.io as mio
from menpo.feature import fast_dsift, igo
from menpo.visualize import print_progress
from menpofit.aam import HolisticAAM
from menpowidgets import visualize_images
from pathlib import Path

def test():
    import menpo.io as mio
    img = mio.import_image('/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/300w_cropped/01_Indoor/indoor_001.png')

    if img.n_channels != 1:
        img = img.as_greyscale()

    img.landmarks['face'] = mio.import_landmark_file('/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/300w_cropped/01_Indoor/indoor_001.pts')

    # objects return copies rather than mutating self, so we can chain calls
    img = (img.crop_to_landmarks(group='face', boundary=10)
           .rescale_landmarks_to_diagonal_range(100, group='face'))

    # now lets take an image feature...
    from menpo.feature import fast_dsift
    img = fast_dsift(img)

    # ...and extract the vector of pixels contained in the
    # convex hull of the face...
    vector = img.as_masked().constrain_mask_to_landmarks(group='face').as_vector()

    print(type(vector), vector.shape)
    # output: <class 'numpy.ndarray'> (3801,)

def fit():
    import menpo.io as mio
    path_to_images = '/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/otherDB/lfpw/trainset'
    training_images = mio.import_images(path_to_images, verbose=True)
    
    %matplotlib inline
    from menpowidgets import visualize_images
    visualize_images(training_images[3])
    
def fit_pre(image_paths):
    import menpo.io as mio
    from menpowidgets import visualize_images
    from menpo.visualize import print_progress

    training_images = []
    for image_path in image_paths:
        for img in print_progress(mio.import_images(image_path, verbose=True)):
            visualize_images(img)
            # convert to greyscale
            if img.n_channels == 3:
                img = img.as_greyscale()
            # crop to landmarks bounding box with an extra 20% padding
            img = img.crop_to_landmarks_proportion(0.2)
            # rescale image if its diagonal is bigger than 400 pixels
            d = img.diagonal()
            if d > 400:
                img = img.rescale(400.0 / d)
#             %matplotlib inline
#             visualize_images(img)
            # append to list
            training_images.append(img)
        
def train_AAM(training_images, feature=igo):
    aam = HolisticAAM(training_images, reference_shape=None,
                      diagonal=150, scales=(0.5, 1.0),
                      holistic_features=feature, verbose=True,
                      max_shape_components=20, max_appearance_components=150)
    print(aam)
    return aam

def fitter_AAM(aam):
    from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional

    fitter = LucasKanadeAAMFitter(aam, lk_algorithm_cls=WibergInverseCompositional,
                                  n_shape=[5, 20], n_appearance=[30, 150])
    return fitter

def pca(path_to_images):

    path_to_lfpw = Path(path_to_images)

    training_shapes = []
    for lg in print_progress(mio.import_landmark_files(path_to_lfpw / '*.pts', verbose=True)):
        training_shapes.append(lg['all'])
    
    %matplotlib inline
    from menpowidgets import visualize_pointclouds
    visualize_pointclouds(training_shapes)

def test_AAM(fitter, test_images, test_gts):
    for test_image in test_images:
        test_gt = test_image.split('.')[0] + '.pts'
        test_image = test_image.as_greyscale()
    
    pass
              
if __name__ == '__main__':
    path_to_images = ['/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/otherDB/lfpw/trainset']
    test_images = ['/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/300w_cropped/01_Indoor/indoor_110.png']                   
    test_gts = ['/home/sean/workplace/221/py-R-FCN-test/data/DB/face/300-w_face/300w_cropped/01_Indoor/indoor_110.pts']                              
    training_images = fit_pre(path_to_images)
    aam = train_AAM(training_images)
    fitter = fitter_AAM(aam)
    print 's'

SyntaxError: invalid syntax (<ipython-input-1-a4485f81c845>, line 88)